In [1]:
import pandas as pd
df = pd.read_csv('C:/Users/CHK/Desktop/Final_Project/data/edu_data_F.csv')
mean_result = df.groupby('SEQ')[['INTERIOR_AM', 'INSUHOS_AM', 'OFFEDU_AM', 'TRVLEC_AM', 
                            'FSBZ_AM', 'SVCARC_AM', 'DIST_AM', 'PLSANIT_AM', 
                            'CLOTHGDS_AM', 'AUTO_AM']].mean().reset_index()

In [ ]:
mean_result.iloc[:, 1:] = mean_result.iloc[:, 1:] * (1/3)

In [2]:
mean_result.to_csv('월 평균 분야별 카드 사용액.csv')

In [3]:
mean_df = pd.read_csv('월 평균 분야별 카드 사용액.csv')

In [4]:
mean_df.head()

,Unnamed: 0,SEQ,INTERIOR_AM,INSUHOS_AM,OFFEDU_AM,TRVLEC_AM,FSBZ_AM,SVCARC_AM,DIST_AM,PLSANIT_AM,CLOTHGDS_AM,AUTO_AM
0,0,00011N2I1MX4X9NQ16C0,0.0,0.00,0.00,0.00,40.000000,0.00,148.571429,0.0,0.00,5.714286
1,1,0003UZ715F1AVTCFVTLJ,0.0,86.25,78.75,6.25,55.000000,11.25,397.500000,5.0,1.25,0.000000
2,2,0005S4MIPCAD7O9L801P,0.0,0.00,0.00,0.00,3.333333,0.00,116.666667,0.0,0.00,0.000000
3,3,000C4EHXNK4O3VQRPMG3,0.0,5.00,0.00,0.00,10.000000,0.00,5.000000,0.0,0.00,0.000000
4,4,000FQ6EU9C0VJG9ECRUV,0.0,0.00,0.00,135.00,3.750000,72.50,2.500000,0.0,0.00,0.000000


In [14]:
# mean_df.drop('Unnamed: 0', axis=1, inplace=True)
mean_df['CAR_PRICE'] = 0
mean_df.columns

Index(['Unnamed: 0', 'SEQ', 'INTERIOR_AM', 'INSUHOS_AM', 'OFFEDU_AM',
       'TRVLEC_AM', 'FSBZ_AM', 'SVCARC_AM', 'DIST_AM', 'PLSANIT_AM',
       'CLOTHGDS_AM', 'AUTO_AM', 'CAR_PRICE'],
      dtype='object')

In [16]:
mean_df = mean_df.drop(columns=['SEQ', 'AUTO_AM'])

In [18]:
mean_df = mean_df.drop(columns=['Unnamed: 0'])

In [30]:
import numpy as np

# Generate random weights for each column
weights = {
    'INTERIOR_AM': np.random.uniform(10, 30),
    'INSUHOS_AM': np.random.uniform(10, 30),
    'OFFEDU_AM': np.random.uniform(10, 30),
    'TRVLEC_AM': np.random.uniform(10, 30),
    'FSBZ_AM': np.random.uniform(10, 30),
    'SVCARC_AM': np.random.uniform(10, 30),
    'DIST_AM': np.random.uniform(10, 30),
    'PLSANIT_AM': np.random.uniform(10, 30),
    'CLOTHGDS_AM': np.random.uniform(10, 30),
}

# Apply weights to each column and calculate CAR_PRICE
mean_df['CAR_PRICE'] = mean_df.drop(columns=['CAR_PRICE']).mul(weights).sum(axis=1)

# Generate random biases for each row
biases = np.random.uniform(-100, 100, size=len(mean_df))

# Add biases to CAR_PRICE
mean_df['CAR_PRICE'] += biases

mean_df['CAR_PRICE'] = mean_df['CAR_PRICE'].apply(lambda x: 0 if x < 50 else x)

print(mean_df.head())

   INTERIOR_AM  INSUHOS_AM  OFFEDU_AM  TRVLEC_AM    FSBZ_AM  SVCARC_AM  \
0          0.0        0.00       0.00       0.00  40.000000       0.00   
1          0.0       86.25      78.75       6.25  55.000000      11.25   
2          0.0        0.00       0.00       0.00   3.333333       0.00   
3          0.0        5.00       0.00       0.00  10.000000       0.00   
4          0.0        0.00       0.00     135.00   3.750000      72.50   

      DIST_AM  PLSANIT_AM  CLOTHGDS_AM     CAR_PRICE  
0  148.571429         0.0         0.00   4429.800923  
1  397.500000         5.0         1.25  14344.894108  
2  116.666667         0.0         0.00   2932.804569  
3    5.000000         0.0         0.00    371.144142  
4    2.500000         0.0         0.00   4309.658519  


In [31]:
mean_df['CAR_PRICE'].describe()

count    892997.000000
mean       5078.928069
std        5487.078789
min           0.000000
25%        1227.669325
50%        3334.909529
75%        6854.223241
max       48481.517060
Name: CAR_PRICE, dtype: float64

In [33]:
len(mean_df[mean_df['CAR_PRICE']==0])

19109

In [34]:
mean_df.columns

Index(['INTERIOR_AM', 'INSUHOS_AM', 'OFFEDU_AM', 'TRVLEC_AM', 'FSBZ_AM',
       'SVCARC_AM', 'DIST_AM', 'PLSANIT_AM', 'CLOTHGDS_AM', 'CAR_PRICE'],
      dtype='object')

# ■ 모델링

In [35]:
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error

# Drop 'SEQ' and 'CAR_PRICE' columns to create X
X = mean_df.drop(columns=['CAR_PRICE'])

# Define the target variable
y = mean_df['CAR_PRICE']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
print("train set:", len(X_train), "|", "test set:", len(X_test))
# Instantiate XGBRegressor
model = xgb.XGBRegressor()

# Train the model
model.fit(X_train, y_train)

# Predict on the test set
y_pred = model.predict(X_test)

# Evaluate the model
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
print("Mean Squared Error:", mse)
print("Mean Squared Error:", mae)

train set: 714397 | test set: 178600
Mean Squared Error: 52760.478680520195
Mean Squared Error: 147.4516526427273


In [48]:
min_range = y_pred - mae
max_range = y_pred + mae

# 각 요소에 대한 최소 오차 범위 계산
min_range_individual = np.where(min_range >= 0, min_range, 0)

is_within_range = np.logical_and(y_pred > min_range, y_pred < max_range)

result = pd.DataFrame({'실제값': y_test,
                       '예측값': y_pred,
                       '최소 오차 범위': min_range_individual,
                       '최대 오차 범위': max_range,
                       '예측값 범위 포함 여부': is_within_range})

result["예측값"] = result["예측값"].astype(int)
result["최소 오차 범위"] = result["최소 오차 범위"].astype(int)
result["최대 오차 범위"] = result["최대 오차 범위"].astype(int)
print(result)
print("오차 범위 내에 포함된 예측의 수", len(result[result['예측값 범위 포함 여부']==True]))
print("오차 범위 내에 포함되지 않은 예측의 수", len(result[result['예측값 범위 포함 여부']!=True]))

                 실제값    예측값  최소 오차 범위  최대 오차 범위  예측값 범위 포함 여부
457038   2399.458549   2241      2094      2389          True
126584   1029.784547   1053       906      1201          True
804004    849.056368    723       575       870          True
373087  20414.584345  19697     19550     19845          True
115772   1832.020454   1962      1814      2109          True
...              ...    ...       ...       ...           ...
146302  13799.849542  13672     13524     13819          True
594586   2384.538078   2333      2186      2480          True
207710  11593.678276  11746     11598     11893          True
822156   4147.175018   4134      3987      4282          True
216127   3963.106643   4088      3941      4236          True

[178600 rows x 5 columns]
오차 범위 내에 포함된 예측의 수 178600
오차 범위 내에 포함되지 않은 예측의 수 0


In [49]:
result.to_csv('모델 결과.csv')